### Load Envs 

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()


### ReAct

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()

#### ReAct Agent
from agents.react.react_agent import ReActAgent

GPT_VER = '0613'
react_agent = ReActAgent(reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=5)

query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2

react_agent.clear_logs()
# answer = react_agent.runnable_agent_step.invoke(query)
# answer = react_agent.runnable_agent_episode.invoke(query, **{'reference':reference})
answer = react_agent.runnable_agent_trial.invoke(query, **{'num_trials':num_trials, 'reference':reference})

### Reflexion ReAct

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()

#### Reflexion Agent
from agents.react.react_agent_with_reflexion import ReActWithReflexionAgent
from agents.react.react_fewshot_examples import REACT_REFLECTION_HEADER, REACT_REFLECTIONS
from chains.reflexion_chains import BaseReflexionChain

GPT_VER = '0613'   
reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/experimental-reflextion", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=REACT_REFLECTIONS
                    )

reflexion_prompt = "jet-taekyo-lee/reflexion-react"
reflexion_agent = ReActWithReflexionAgent(
                        reflexion_chain = reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=5)


query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2

reflexion_agent.clear_logs()
# answer = reflexion_agent.runnable_agent_step.invoke(query)
# answer = reflexion_agent.runnable_agent_episode.invoke(query, **{'reference':reference})
answer = reflexion_agent.runnable_agent_trial.invoke(query, **{'num_trials':num_trials, 'reference':reference})



### OpenAI Regacy agent

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()

# #### OepnAI Agent
GPT_VER = '0613' 
from agents.openai_legacy.openai_legacy_agent import OpenAILegacyAgent
openai_agent = OpenAILegacyAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = "jet-taekyo-lee/openai-function-calling-agent",
    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
    action_word = 'Invoking',
    horizon = 5)


query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2

openai_agent.clear_logs()
# answer = openai_agent.runnable_agent_step.invoke(query)
# answer = openai_agent.runnable_agent_episode.invoke(query, **{'reference':reference})
answer = openai_agent.runnable_agent_trial.invoke(query, **{'num_trials':num_trials, 'reference':reference})



### Reflexion OpenAI Legacy agent

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()

# # #### Reflexion OepnAI Agent
from agents.openai_legacy.openai_legacy_with_reflexion_agent import OpenAILegacyWithReflexionAgent
from agents.openai_legacy.openai_legacy_fewshot_examples import OPENAI_LEGACY_REFLECTIONS, OPENAI_LEGACY_REFLECTION_HEADER
from chains.reflexion_chains import BaseReflexionChain


GPT_VER = '0613'    
openai_legacy_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_LEGACY_REFLECTIONS
                    )


openai_legacy_reflexion_agent = OpenAILegacyWithReflexionAgent(
                        reflexion_chain = openai_legacy_reflexion_chains,
                        reflexion_header = OPENAI_LEGACY_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/rexlexion-openai-function-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word='Invoking',
                        file_logging=True,
                        horizon = 5)


query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2

openai_legacy_reflexion_agent.clear_logs()
# answer = openai_legacy_reflexion_agent.runnable_agent_step.invoke(query)
# answer = openai_legacy_reflexion_agent.runnable_agent_episode.invoke(query, **{'reference':reference})
answer = openai_legacy_reflexion_agent.runnable_agent_trial.invoke(query, **{'num_trials':num_trials, 'reference':reference})



### Parallel Function Calling agent

In [4]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
import asyncio
config = HD22Configuration()

#### Parallel Function Calling Agent
GPT_VER = '1106' 
from agents.openai_parallel.openai_parallel_agent import OpenAIParallelAgent

openai_parallel_agent = OpenAIParallelAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
    action_word = 'Tool invocations in parallel',
    observation_word = 'Tool-invocations results in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
    horizon = 5
)



query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2


# openai_parallel_agent.clear_logs()

# await openai_parallel_agent.runnable_agent_step.ainvoke(query)
# await openai_parallel_agent.runnable_agent_episode.ainvoke(query, **{'reference':reference})
await openai_parallel_agent.runnable_agent_trial.ainvoke(query, **{'reference':reference, 'num_trials':num_trials})



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of Korea age), Tool 2-> YDCSearch(query=MLB American League MVP last season age)]
[INFO] (step 1-observation) :: [Tool 1-> The current former president of South Korea is Moon Jae-in, who was born on January 24, 1953. As of my knowledge cutoff in 2023, Moon Jae-in would be 70 years old., Tool 2-> The American League MVP for the 2022 MLB season was Aaron Judge of the New York Yankees. Aaron Judge was born on April 26, 1992, which made him 30 years old during the 2022 season when he won the MVP award.]
[INFO] (step 2-action) Tool invocations in parallel: Now I can answer directly without resorting to any tool.
[INFO] Answer: Moon Jae-in, the former president of Korea, is older than Aaron Judge, the MLB American League 

RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 

In [ ]:
# import os
# from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
# from tools.get_tools import DefaultSchemasTools
# from langchain import hub
# from utils.load_envs import Environ
# from chains.qa_evaluators import CustomQAEvaluator
# from configs.hd22_config import HD22Configuration
# import asyncio
# config = HD22Configuration()

# #### Parallel Function Calling Agent
# GPT_VER = '1106' 
# from agents.openai_parallel.openai_parallel_agent import OpenAIParallelAgent

# openai_parallel_agent = OpenAIParallelAgent(
#     reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
#     base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
#     schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
#     evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
#     action_word = 'Tool invocations in parallel',
#     observation_word = 'Tool-invocations results in parallel',
#     use_chat_completion_api = True,
#     azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
#     horizon = 5
# )



# query = "Did the Yankees win more than 100 games last season?"
# reference = "No, They did not win more than 100 games last season."
# num_trials = 2


# # openai_parallel_agent.clear_logs()

# # await openai_parallel_agent.runnable_agent_step.ainvoke(query)
# # await openai_parallel_agent.runnable_agent_episode.ainvoke(query, **{'reference':reference})
# await openai_parallel_agent.runnable_agent_trial.ainvoke(query, **{'reference':reference, 'num_trials':num_trials})



### Reflexion + Parallel Function Calling agent

In [ ]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio
config = HD22Configuration()

#### Reflexion + Parallel Function Calling Agent
from chains.reflexion_chains import BaseReflexionChain
from agents.openai_parallel.openai_parallel_with_reflexion_agent import OpenAIParallelWithReflexionAgent
from agents.openai_parallel.openai_parallel_fewshot_examples import OPENAI_PARALLEL_REFLECTIONS, OPENAI_PARALLEL_REFLECTION_HEADER

GPT_VER = '1106'    
openai_parallel_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-parallel-calling-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_PARALLEL_REFLECTIONS
                    )


openai_parallel_reflexion_agent = OpenAIParallelWithReflexionAgent(
                        reflexion_chain = openai_parallel_reflexion_chains,
                        reflexion_header = OPENAI_PARALLEL_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/reflexion-openai-parallel-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word = 'Tool invocations in parallel',
                        observation_word = 'Tool-invocations results in parallel',
                        use_chat_completion_api = True,
                        azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
                        file_logging=True,
                        horizon = 5)


query = "Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?"
reference = "The former former president of Korea is older. The sum of their age is 99."
num_trials = 2



openai_parallel_reflexion_agent.clear_logs()
# await openai_parallel_reflexion_agent.runnable_agent_step.ainvoke(query)
# await openai_parallel_reflexion_agent.runnable_agent_episode.ainvoke(query, **{'reference':reference})
await openai_parallel_reflexion_agent.runnable_agent_trial.ainvoke(query, **{'reference':reference, 'num_trials':num_trials})

